# Light GBM

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2007-01-01'
end = '2018-11-16'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-01-03,20.080000,20.400000,19.350000,19.520000,19.520000,28350300
2007-01-04,19.660000,19.860001,19.320000,19.790001,19.790001,23652500
2007-01-05,19.540001,19.910000,19.540001,19.709999,19.709999,15902400
2007-01-08,19.709999,19.860001,19.370001,19.469999,19.469999,15814800
2007-01-09,19.450001,19.709999,19.370001,19.650000,19.650000,14494200


In [3]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,
2007-01-04,19.660000,19.860001,19.320000,19.790001,19.790001,23652500,0,0,0,0.013832
2007-01-05,19.540001,19.910000,19.540001,19.709999,19.709999,15902400,0,1,0,-0.004043
2007-01-08,19.709999,19.860001,19.370001,19.469999,19.469999,15814800,0,0,1,-0.012177
2007-01-09,19.450001,19.709999,19.370001,19.650000,19.650000,14494200,1,1,1,0.009245
2007-01-10,19.639999,20.020000,19.500000,20.010000,20.010000,19783200,1,1,1,0.018321


In [4]:
dataset.shape

(2991, 10)

In [5]:
X = dataset[['Open', 'High', 'Low', 'Volume']].values
y = dataset['Buy_Sell'].values

In [9]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [13]:
model = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
model.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)

[1]	valid_0's l2: 0.249178	valid_0's l1: 0.498865
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.248777	valid_0's l1: 0.498389
[3]	valid_0's l2: 0.248759	valid_0's l1: 0.498251
[4]	valid_0's l2: 0.248304	valid_0's l1: 0.497638
[5]	valid_0's l2: 0.248418	valid_0's l1: 0.497581
[6]	valid_0's l2: 0.248304	valid_0's l1: 0.497318
[7]	valid_0's l2: 0.248443	valid_0's l1: 0.497276
[8]	valid_0's l2: 0.248523	valid_0's l1: 0.497175
[9]	valid_0's l2: 0.248315	valid_0's l1: 0.496795
[10]	valid_0's l2: 0.248275	valid_0's l1: 0.496562
[11]	valid_0's l2: 0.248447	valid_0's l1: 0.496506
[12]	valid_0's l2: 0.248285	valid_0's l1: 0.496182
[13]	valid_0's l2: 0.248401	valid_0's l1: 0.496052
[14]	valid_0's l2: 0.248347	valid_0's l1: 0.495758
[15]	valid_0's l2: 0.2484	valid_0's l1: 0.495516
Early stopping, best iteration is:
[10]	valid_0's l2: 0.248275	valid_0's l1: 0.496562


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.05, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=20, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [14]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration_)

In [16]:
from sklearn.metrics import mean_squared_error
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 0.498272504454


In [18]:
print('LightGBM Score:', model.score(X_test, y_test))

LightGBM Score: 0.00423101252743
